## Data Ingest and Cleaning

In [145]:
import geopandas as gpd
import pandas as pd

----

We started by trying to connect to the publicly available API endpoint the colorado information marketplace lists for the geojson data. We didn't have to sign up for any credentials but later noticed that the result only brought down 1000 records and it was clear the dataset extended around 3.5x that amount.

```python
# healthcare_url = 'https://opendata.arcgis.com/datasets/914bc3a28a644f95b13829128e69ede4_0.geojson'
# healthcare_df = gpd.read_file(healthcare_url)```


We changed strategy to just download the zipped shapefile to */data/input/* and work from there as it included the full dataset and geopandas can just read directly from the zipped file.

```python
healthcare_zip = "zip://../../data/input/CDPHE_Health_Facilities.zip"
healthcare_df = gpd.read_file(healthcare_zip)```

In [146]:
# get healthcare facilities

#these were only reading 1000 rows for some reason. Changed to reading in from zipped file
# healthcare_url = 'https://opendata.arcgis.com/datasets/914bc3a28a644f95b13829128e69ede4_0.geojson'
# healthcare_df = gpd.read_file(healthcare_url)

healthcare_zip = "zip://../../data/input/CDPHE_Health_Facilities.zip"
healthcare_df = gpd.read_file(healthcare_zip)

In [147]:
healthcare_df.dtypes

OBJECTID         int64
FAC_NAME        object
NAME            object
ADDRESS         object
CITY_STATE      object
CITY            object
STATE           object
ZIP             object
OPERATING_      object
MEDICARE        object
MEDICARE_I      object
MEDICAID        object
MEDICAID_I      object
CERTIFIED_      object
COUNTY          object
ADMIN           object
PHONE           object
EMERGENCY_      object
EMERGENC_1      object
EMERGENC_2      object
EMERGENC_3      object
DATE_GEOCO      object
SOURCE          object
GEO_TRACT       object
GEO_COUNTY      object
GEO_ZIP         object
FACTYPE         object
SYMBOL          object
Latitude       float64
Longitude      float64
TYPE            object
geometry      geometry
dtype: object

In [148]:
healthcare_df.head(5)

,OBJECTID,FAC_NAME,NAME,ADDRESS,CITY_STATE,CITY,STATE,ZIP,OPERATING_,MEDICARE,...,SOURCE,GEO_TRACT,GEO_COUNTY,GEO_ZIP,FACTYPE,SYMBOL,Latitude,Longitude,TYPE,geometry
0,3001,HUMAN TOUCH UNSKILLED SERVICES PUEBLO INC (04L...,HUMAN TOUCH UNSKILLED SERVICES PUEBLO INC,1008 W ABRIENDO AVE,"PUEBLO, CO 81004",PUEBLO,CO,81004,01-ACTIVE,N,...,CDPHE Health Facilities and Emergency Medical ...,08101001500,PUEBLO,81004,Home Care Agency - In-Home Support Services,Home Health,38.268394,-104.632767,HCA-IHSS,POINT (-104.63277 38.26839)
1,3002,BROOKDALE EL CAMINO (2306MT),BROOKDALE EL CAMINO,4723 SURFWOOD LANE,"PUEBLO, CO 81005",PUEBLO,CO,81005,01-ACTIVE,N,...,CDPHE Health Facilities and Emergency Medical ...,08101002801,PUEBLO,81005,Assisted Living Residence,Assisted Living Residence/Nursing Home,38.228497,-104.671387,ALRONLY,POINT (-104.67139 38.22850)
2,3003,GOLDEN HORIZON (230663),GOLDEN HORIZON,2109 CHAUTARD,"PUEBLO, CO 81005",PUEBLO,CO,81005,01-ACTIVE,N,...,CDPHE Health Facilities and Emergency Medical ...,08101002801,PUEBLO,81005,Assisted Living Residence,Assisted Living Residence/Nursing Home,38.228157,-104.668221,ALR/ACF,POINT (-104.66822 38.22816)
3,3004,LIFE CARE CENTER OF PUEBLO (020641),LIFE CARE CENTER OF PUEBLO,2118 CHATALET LN,"PUEBLO, CO 81005",PUEBLO,CO,81005,01-ACTIVE,Y,...,CDPHE Health Facilities and Emergency Medical ...,08101002801,PUEBLO,81005,Nursing Home,Assisted Living Residence/Nursing Home,38.226883,-104.665939,SNF/NF,POINT (-104.66594 38.22688)
4,3005,PUEBLO COMMUNITY CONNECTIONS (10F366),PUEBLO COMMUNITY CONNECTIONS,3913 SANDLEWOOD LANE,"PUEBLO, CO 81005",PUEBLO,CO,81005,01-ACTIVE,N,...,CDPHE Health Facilities and Emergency Medical ...,08101002700,PUEBLO,81005,Home and Community Based Services - Intellectu...,Home and Community Based Services,38.231850,-104.660072,HCBS-IDD,POINT (-104.66007 38.23185)


In [149]:
print('Facility Types included in this dataset:')
healthcare_df.FACTYPE.unique().tolist()

Facility Types included in this dataset:


['Home Care Agency - In-Home Support Services',
 'Assisted Living Residence',
 'Nursing Home',
 'Home and Community Based Services - Intellectual and developmental disabilities',
 'Ambulatory Surgical Center',
 'Home Health Agency - Medicare/Medicaid',
 'Home Health Agency - HCA license - personal care homemaker',
 'Hospice (Medicare)',
 'Hospice (Licensed)',
 'Residential Care Facility for persons with developmental disabilities',
 'Home Health Care - Persons with intellectual and developmental disabilities',
 'Community Clinic/Emergency',
 'End Stage Renal Disease Facility',
 'Outpatient Physical Therapy/Speech Pathology',
 'Portable X-Ray',
 'Home Health Agency - HCA license only',
 'Federally Qualified Health Center',
 'Critical Access Hospital',
 'Rural Health Clinic',
 'Community Mental Health Clinic',
 'Department of Corrections Clinic',
 'Hospital - non participating',
 'Adult Day',
 'Hospital',
 'Home Health Agency - Licensed only',
 'Acute Treatment Unit',
 'Community Clinic'

-----
-----

It became clear that there are many different facility types listed in this dataset ranging from home health to hospice. Because this analysis is targeted to COVID-19 infections, we went through the list and identified facility types that seemed likely to be administering care for this specific disease.

-----
-----



In [150]:
#identifying facility types of interest during the COVID-19 pandemic
keep_hc = [
    'Federally Qualified Health Center', 'Hospital', 'Rural Health Clinic',
    'Community Clinic/Emergency', 'Critical Access Hospital',
    'Hospital - non participating', 'Community Clinic', 'Long Term Hospital',
    'Acute Treatment Unit', "Children's Hospital"]

#slicing to keep only facilities of interest as identified in keep_hc
healthcare_df = healthcare_df[healthcare_df.FACTYPE.isin(keep_hc)]

-----
-----

After narrowing the options, we can look at the frequency of each facility type among the remaining data.

-----
-----

In [151]:
print('\n\nFacility Type                       |Frequency')
healthcare_df.FACTYPE.value_counts()



Facility Type                       |Frequency


Federally Qualified Health Center    118
Hospital                              56
Rural Health Clinic                   51
Community Clinic/Emergency            43
Critical Access Hospital              32
Hospital - non participating           9
Community Clinic                       9
Acute Treatment Unit                   6
Long Term Hospital                     6
Children's Hospital                    2
Name: FACTYPE, dtype: int64

### Narrowing in on Communities Experiencing Inequity

At this point, we needed to bring in data containing race/ethnicity by some geometry. We went for the finest available geometries, blocks, sourced from the American Community Survey 2018 results for Colorado. This was also sourced on the Colorado Information Marketplace. Again, we started by using the API endpoint but quickly realized it was limiting to 1000 rows so we downloaded the shapefile to */data/input/ as well and read it in use geopandas.

In [152]:
#get ACS data

# This was limiting to 1000 rows for some reason. Changed to reading from zipped file
# ACS_url = 'https://data.colorado.gov/resource/ge9s-ra8y.geojson'
# ACS_df = gpd.read_file(ACS_url)

acs_zip = "zip://../../data/input/Census Block Groups in Colorado 2018.zip"
ACS_df = gpd.read_file(acs_zip)

-----
-----

There are a ton of fields in the ACS data that were interesting, but not a focus of this analysis. We narrowed it down to the geometry, unique id's for the blocks, and race/ethnicity data. 

The Office of Health Equity put out fact sheets (see ../../resources/cdphe-ohe/). They specifically test for health inequities among each of these populations. However, they also test when aggregated by all non-white persons in Colorado. We created indexed rates of that view as well as percentage based rates for those as well. 

-----
-----

In [153]:
ACS_df.head(5)

,age10_14,age15_19,age18_24,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,...,v750k_1m,v_1m_plus,v_l_50k,vac_hu,w_16pl_nh,walk,white_nh,wrk_home,wrkrs_16pl,geometry
0,65.0,53.0,64.0,46.0,23.0,9.0,25.0,27.0,60.0,73.0,...,0.0,0.0,0.0,37.0,None,None,581.0,None,None,"POLYGON ((-102.39831 40.57112, -102.30768 40.5..."
1,16.0,44.0,21.0,13.0,33.0,10.0,14.0,15.0,64.0,101.0,...,0.0,15.0,0.0,0.0,None,None,622.0,None,None,"POLYGON ((-104.75122 38.91043, -104.74620 38.9..."
2,33.0,45.0,36.0,34.0,42.0,80.0,16.0,18.0,59.0,49.0,...,3.0,0.0,12.0,273.0,None,None,766.0,None,None,"POLYGON ((-105.02795 38.93003, -105.02788 38.9..."
3,162.0,70.0,23.0,23.0,97.0,180.0,72.0,205.0,25.0,86.0,...,0.0,0.0,187.0,0.0,None,None,383.0,None,None,"POLYGON ((-104.79750 38.79603, -104.79441 38.7..."
4,0.0,53.0,60.0,20.0,40.0,46.0,46.0,0.0,81.0,60.0,...,0.0,0.0,0.0,0.0,None,None,799.0,None,None,"POLYGON ((-104.74796 38.87632, -104.74509 38.8..."


In [154]:
ACS_df.columns.tolist()

['age10_14',
 'age15_19',
 'age18_24',
 'age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',
 'age45_49',
 'age50_54',
 'age55_59',
 'age5_9',
 'age60_64',
 'age65_69',
 'age70_74',
 'age75_79',
 'age80_84',
 'age85pl',
 'ageless18',
 'ageless5',
 'armedfrcs',
 'asian_nh',
 'avghhsize',
 'b1939_e',
 'b1940_1949',
 'b1950_1959',
 'b1960_1969',
 'b1970_1979',
 'b1980_1989',
 'b1990_1999',
 'b2000_2009',
 'bachl_hghr',
 'bike',
 'black_nh',
 'blt_2010_p',
 'born_in_co',
 'brn_oth_st',
 'car_all',
 'car_alone',
 'car_carpoo',
 'citz_birth',
 'citz_nat',
 'civ_lf',
 'civ_ni_',
 'civ_ni_pop',
 'diff_state',
 'disabled',
 'emp',
 'enrolled',
 'familyhh',
 'female',
 'foreign_b',
 'frm_abroad',
 'geoname',
 'geonum',
 'gr_1_4',
 'gr_5_8',
 'gr_9_12',
 'grad_prof',
 'hawpi_nh',
 'hhi100_125',
 'hhi125_150',
 'hhi150_200',
 'hhi200_pl',
 'hhi20_30',
 'hhi30_40',
 'hhi40_50',
 'hhi50_60',
 'hhi60_75',
 'hhi75_100',
 'hhi_l20k',
 'hhldr_naln',
 'hhldralone',
 'hispanic',
 'households',


In [155]:
#define fields to keep in ACS_df
ACS_df = ACS_df[['asian_nh', 'hawpi_nh','ntvam_nh','other_nh','black_nh','white_nh','pop','geonum','geoname','geometry']]

In [156]:
ACS_df.iloc[:,0:5]#.sum(axis=1)

,asian_nh,hawpi_nh,ntvam_nh,other_nh,black_nh
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,10.0
2,25.0,0.0,0.0,0.0,0.0
3,28.0,0.0,5.0,0.0,96.0
4,17.0,0.0,0.0,0.0,24.0
...,...,...,...,...,...
3527,140.0,0.0,4.0,0.0,0.0
3528,20.0,0.0,0.0,0.0,0.0
3529,52.0,0.0,79.0,0.0,0.0
3530,98.0,0.0,9.0,0.0,31.0


In [157]:
#updating types in preparation for calculations to come
def convert_type_inplace(gdf, col, dtype):
    '''converts column in place given the frame, column, and desired data type'''
    try:
        gdf[col] = gdf[col].astype(dtype)
    except:
        print('Error: Unable to convert',col, 'to type', dtype)
    return str(col)+'converted to '+str(type)

#loop through cols to convert to float and make conversion
for colname in ['asian_nh','hawpi_nh','ntvam_nh', 'other_nh', 'black_nh','white_nh','pop']:
    convert_type_inplace(ACS_df, colname, 'float')
    
#add column for unknown persons. Some residents seem to have no race/ethnicity reported so we address that as well
ACS_df['unknown_nh'] = ACS_df['pop'] - ACS_df.iloc[:,0:6].sum(axis=1)

#calculate non-white household count considering unknown race/ethnicities
ACS_df['non_white_nh'] = ACS_df['pop'] - ACS_df['white_nh'] - ACS_df['unknown_nh']

ACS_df.head()

,asian_nh,hawpi_nh,ntvam_nh,other_nh,black_nh,white_nh,pop,geonum,geoname,geometry,unknown_nh,non_white_nh
0,0.0,0.0,0.0,0.0,0.0,581.0,593.0,1.080960e+12,1,"POLYGON ((-102.39831 40.57112, -102.30768 40.5...",12.0,0.0
1,0.0,0.0,0.0,0.0,10.0,622.0,787.0,1.080410e+12,2,"POLYGON ((-104.75122 38.91043, -104.74620 38.9...",155.0,10.0
2,25.0,0.0,0.0,0.0,0.0,766.0,846.0,1.080410e+12,3,"POLYGON ((-105.02795 38.93003, -105.02788 38.9...",55.0,25.0
3,28.0,0.0,5.0,0.0,96.0,383.0,1252.0,1.080410e+12,1,"POLYGON ((-104.79750 38.79603, -104.79441 38.7...",740.0,129.0
4,17.0,0.0,0.0,0.0,24.0,799.0,874.0,1.080410e+12,1,"POLYGON ((-104.74796 38.87632, -104.74509 38.8...",34.0,41.0


In [158]:
def calculate_index(gdf, target_col, total_col):
    '''calculates indexed concentration of target column among total column per geometry'''
    
    try:
        #calculate sum of population across state and among target population
        popsum = gdf[total_col].sum()
        groupsum = gdf[target_col].sum()
    
        #calculate index, or ratio of target pop among total pop in geometry relative to state ratio.
        gdf[(target_col+'_index')] = ((gdf[target_col] / gdf[total_col]) / (groupsum / popsum))
        
    except:
        print('unable to add column:'+ target_col +'_index to '+ gdf)

        
        
#calculate index for each population compared to colorado in general and append to ACS_df
for target_population in ['asian_nh','hawpi_nh','ntvam_nh', 'other_nh', 'black_nh','white_nh', 'non_white_nh', 'unknown_nh']:
    calculate_index(ACS_df, target_population, 'pop')

def calculate_percentage(gdf, target_col, total_col):
    '''calculates percentage composition target column among total population per geometry'''
    try:
        #calculate percentage of target column among total column
        gdf[(target_col+'_pct')] = gdf[target_col] / gdf[total_col]
    except:
        print('unable to add column:'+ target_col +'_pct to '+ gdf)
    
for target_population in ['asian_nh','hawpi_nh','ntvam_nh', 'other_nh', 'black_nh','white_nh', 'non_white_nh', 'unknown_nh']:
    calculate_percentage(ACS_df, target_population, 'pop')  
    
ACS_df = ACS_df.set_index(ACS_df['geonum'])
ACS_df.head()

,asian_nh,hawpi_nh,ntvam_nh,other_nh,black_nh,white_nh,pop,geonum,geoname,geometry,...,non_white_nh_index,unknown_nh_index,asian_nh_pct,hawpi_nh_pct,ntvam_nh_pct,other_nh_pct,black_nh_pct,white_nh_pct,non_white_nh_pct,unknown_nh_pct
geonum,,,,,,,,,,,,,,,,,,,,,
1.080960e+12,0.0,0.0,0.0,0.0,0.0,581.0,593.0,1.080960e+12,1,"POLYGON ((-102.39831 40.57112, -102.30768 40.5...",...,0.000000,0.084879,0.000000,0.0,0.000000,0.0,0.000000,0.979764,0.000000,0.020236
1.080410e+12,0.0,0.0,0.0,0.0,10.0,622.0,787.0,1.080410e+12,2,"POLYGON ((-104.75122 38.91043, -104.74620 38.9...",...,0.161845,0.826101,0.000000,0.0,0.000000,0.0,0.012706,0.790343,0.012706,0.196950
1.080410e+12,25.0,0.0,0.0,0.0,0.0,766.0,846.0,1.080410e+12,3,"POLYGON ((-105.02795 38.93003, -105.02788 38.9...",...,0.376395,0.272689,0.029551,0.0,0.000000,0.0,0.000000,0.905437,0.029551,0.065012
1.080410e+12,28.0,0.0,5.0,0.0,96.0,383.0,1252.0,1.080410e+12,1,"POLYGON ((-104.79750 38.79603, -104.79441 38.7...",...,1.312379,2.479153,0.022364,0.0,0.003994,0.0,0.076677,0.305911,0.103035,0.591054
1.080410e+12,17.0,0.0,0.0,0.0,24.0,799.0,874.0,1.080410e+12,1,"POLYGON ((-104.74796 38.87632, -104.74509 38.8...",...,0.597512,0.163171,0.019451,0.0,0.000000,0.0,0.027460,0.914188,0.046911,0.038902


In [159]:
#write output to geoJSON to pick up for visualization
healthcare_df.to_file('../../data/output/healthcare.geojson', driver='GeoJSON')
ACS_df.to_file('../../data/output/ACS.geojson', driver='GeoJSON')

#export geometry of ACS data alone for folium mapping
ACS_df[['geometry',
        'geonum']].to_file('../../data/output/ACS_geometry_only.geoJSON',
                           driver='GeoJSON')

#export csv with results but no geometry for folium mapping
pd.DataFrame(ACS_df[[
    'geonum', 'pop', 'asian_nh', 'hawpi_nh', 'ntvam_nh', 'other_nh',
    'black_nh', 'white_nh', 'unknown_nh', 'non_white_nh', 'asian_nh_index',
    'hawpi_nh_index', 'ntvam_nh_index', 'other_nh_index', 'black_nh_index',
    'white_nh_index', 'non_white_nh_index', 'asian_nh_pct', 'hawpi_nh_pct',
    'ntvam_nh_pct', 'other_nh_pct', 'black_nh_pct', 'white_nh_pct',
    'non_white_nh_pct', 'unknown_nh_index', 'unknown_nh_pct'
]]).to_csv('../../data/output/ACS_data_only.csv', index=False)

In [160]:
#export geometries where black_indexed is > 1
index_101_500 = ACS_df[(ACS_df.black_indexed > 1)
                       & (ACS_df.black_indexed <= 5)]
index_500_1000 = ACS_df[(ACS_df.black_indexed > 5)
                        & (ACS_df.black_indexed <= 10)]
index_1000_1300 = ACS_df[(ACS_df.black_indexed > 10)
                         & (ACS_df.black_indexed <= 13)]
index_1301_plus = ACS_df[ACS_df.black_indexed > 13]

# #export to /data/output
index_101_500.to_file('../../data/output/index_101_500.geojson',
                      driver='GeoJSON')
index_500_1000.to_file('../../data/output/index_500_1000.geojson',
                       driver='GeoJSON')
index_1000_1300.to_file('../../data/output/index_1000_1300.geojson',
                        driver='GeoJSON')
index_1301_plus.to_file('../../data/output/index_1301_plus.geojson',
                        driver='GeoJSON')

#print for sanity check
print('Total number of blocks in original data:', len(ACS_df),
      '\nNumber of blocks 101 to 500% index:',
      len(index_101_500), '\nNumber of blocks 501 - 1000% index:',
      len(index_500_1000), '\nNumber of blocks 1000 - 1300% index:',
      len(index_1000_1300), '\nNumber of blocks 1300+% index:',
      len(index_1301_plus))

AttributeError: 'GeoDataFrame' object has no attribute 'black_indexed'